# API

In [1]:
# Fetches the latest cryptocurrency listings from CoinMarketCap's API using a session with set parameters and headers.
# Handles potential connection issues.


from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
parameters = {
  'start':'1',
  'limit':'50',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [7]:
type(data)

In [3]:
import pandas as pd


#This allows us to see all the columns, not just like 50
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [6]:
#This normalizes the data and makes it all pretty in a dataframe

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.todatetime('now')
df

In [ ]:
# This function retrieves the latest cryptocurrency data from the CoinMarketCap API, normalizes it into a DataFrame, and appends it to a CSV file with a timestamp.

def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    parameters = {
      'start':'1',
      'limit':'50',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.todatetime('now')
    df
 
    if not os.path.isfile(r'C:\Users\aryob\Documents\Bilquiss College\Data Analyst Bootcamp\API.csv'):
        df.to_csv(r'C:\Users\aryob\Documents\Bilquiss College\Data Analyst Bootcamp\API.csv', header='column_names')
    else:
        df.to_csv(r'C:\Users\aryob\Documents\Bilquiss College\Data Analyst Bootcamp\API.csv', mode='a', header=False)

In [ ]:
# This loop runs the api_runner function 333 times, with a 60-second delay between each run, and exits when complete.

import os 
from time import time 
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner Completed Successfully')
    sleep(60)
exit()

In [ ]:
df22 = pd.read_csv(r'C:\Users\aryob\Documents\Bilquiss College\Data Analyst Bootcamp\API.csv')
df22

In [ ]:
# change scientific notation to numbers
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df

In [ ]:
# Now let's look at the coin trends over time

df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3

In [ ]:
df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
df5.count()

In [ ]:
index = pd.Index(range(90))

# Set the above DataFrame index object as the index
df6 = df5.set_index(index)
df6

In [ ]:
# Change the column name

df7 = df6.rename(columns={'level_1': 'percent_change'})
df7

In [ ]:
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['24h','7d','30d','60d','90d'])
df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')

In [ ]:
# we are going to create a dataframe with the columns we want

df10 = df[['name','quote.USD.price','timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10

In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp', y='quote.USD.price', data = df10)